# Which _files_ are in my project?
### Overview
Here we focus on listing all files within a single project. Importantly, files are only accessible **within** a project<sup>1</sup>. As with any **list**-type call, we will get minimal information about each file. We will show two options (A & B) of getting the same information.

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. Your project needs to have at least one file inside
 
### Imports and Definitions
 A single call is sufficient to get a file list. We will show two different options, both of which are defined in the apimethods.py file.
 
 <sup>1</sup> There is no "list **all** *files*" API call that parallels the "list **all**" calls for *projects* and *apps*

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
A **list**-call for projects returns the following *attributes*:
* **id**     _Unique_ identifier for each file, generated by the CGC
* **name**   Name of file, maybe _non-unique_
* **href**   Address<sup>1</sup> of the file.

<sup>1</sup> This *address* is for the API, but will not work in a browser.

In [ ]:
# [USER INPUT] Set project index here, (pick one with files):
p_index = 4

existing_projects = API('projects')                              # LIST all projects

# list the files in the first project
my_files = API(path='files', query={'project': existing_projects.id[p_index]})

# print up to the first 10 files
for ii in range(min(10,len(my_files.id))):
    print('file name is (%s); \t file id is (%s)' % (my_files.name[ii], my_files.id[ii]) )

## (Option B) Use api_call() *function*
Using the api_call() function returns the same information as above, but now as *keys* within a dictionary for each project. To start, we list the first 25 projects you are a member of. Then, we query the _file list_ for the first project. Here we are dealing with pagination manually and purposefully returning only 3 files at a time. 

Additionally, we have adding the _limit_ and _offset_ arguments. In this example only 3 records will be
returned, starting from the first record (index = 0). This requires **pagination**<sup>2</sup>. In the next example we
call the next records (but do not explicited redefine the default arguments).

<sup>2</sup> Note the API() object call automatically manages pagination and will return *all* projects

In [ ]:
# [USER INPUT] Set project index here, (pick one with files):
p_index = 4

# List projects (n=25) I am a member of
existing_projects = api_call(path='projects', method='GET', \
                            query={'limit':25, 'offset':0}, data=None)      

# Short list (n=3) of files within this project
my_files = api_call(path='files', \
                    query={'limit':3, 'project': existing_projects['items'][p_index]['id']})
print(my_files['items'])

# Using pagination to see the next short (n=whatever_first_limit_was) list of projects
if my_files['links'][0]['rel'] == 'next':
    next_path = my_files['links'][0]['href'].split('/v2/')[-1]
    print('\n Query for the next page of results is %s' % (next_path))
    more_files = api_call(path=next_path)
    
    print('We paged to find more files: \n')
    print(more_files['items'])

For the pagination above, we used the 'links' key. 'links' is another dictionary including:
* 'href' is the full path for the **next** api_call. It will have an limit equal to the prior call and a offset
based on the prior call<sup>3</sup>
* 'method' this will be 'GET' as in the prior call
* 'rel' this is the relative direction of the call. If you have reached the end of the list, this will be 'prev'

<sup>3</sup> Note that these query keys (limit, value, offset, etc) can be either passed within the *query* variable or concantenated to the *path* variable after a "?"

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/list-all-your-projects)